In [1]:
%matplotlib inline
# Load all necessary packages
import sys
import pandas as pd
# Import IBM's AI Fairness tooolbox
from aif360.datasets      import BinaryLabelDataset
from aif360.metrics       import BinaryLabelDatasetMetric
from aif360.metrics       import ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector
from aif360.algorithms.preprocessing.lfr import LFR
# Import scikit-learn core slibraries
from sklearn.ensemble      import RandomForestClassifier
from sklearn.linear_model  import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics       import accuracy_score
from IPython.display import Markdown, display
import matplotlib.pyplot as plt
# Warnings will be used to silence various model warnings for tidier output
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read the cleaned Taiwan-Credit-Card-Cleaned-Marriage dataset
Taiwan_df = pd.read_csv('./input/Taiwan-Credit-Card-Cleaned-Marriage.csv')
Taiwan_df.head(3)

,LIMIT_BAL,GENDER,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,DEFAULT
0,20000.0,2.0,2.0,1.0,24.0,2.0,2.0,-1.0,-1.0,-2.0,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,0.0
1,120000.0,2.0,2.0,0.0,26.0,-1.0,2.0,0.0,0.0,0.0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,0.0
2,90000.0,2.0,2.0,0.0,34.0,0.0,0.0,0.0,0.0,0.0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,1.0


In [3]:
# Set privileged (1)/ unprivileged (0)/ favourable (1) / unfavourable values (0)
protected_attr      = 'MARRIAGE'
priv_grp            = 1  # Married males and females 
unpriv_grp          = 0  # Single males and females  
lab                 = 'DEFAULT'
fav_label           = 1 # Will not default next month
unfav_label         = 0 # Will default next month
privileged_groups   = [{protected_attr: priv_grp}]   # Married folk (male and female)
unprivileged_groups = [{protected_attr: unpriv_grp}] # Single folk  (male and female)

In [4]:
# Create a Binary Label Dataset to use with AIF360 APIs
X = Taiwan_df.drop(lab,axis=1)
y = Taiwan_df[lab]
Taiwan_bld = BinaryLabelDataset(df=pd.concat((X, y), axis=1),
                                label_names=[lab], protected_attribute_names=[protected_attr],
                                favorable_label=fav_label, unfavorable_label=unfav_label)

In [5]:
# Create train and test datasets
Taiwan_train_df, Taiwan_test_df = Taiwan_bld.split([0.8], shuffle=True, seed=101)

In [6]:
scaler = MinMaxScaler(copy=False)
Taiwan_train_df.features = scaler.fit_transform(Taiwan_train_df.features)
Taiwan_test_df.features  = scaler.fit_transform(Taiwan_test_df.features)

In [7]:
# Determine the baseline model accuracy for Logistic Regression and Random Forest Classifiers
Taiwan_train, d = Taiwan_train_df.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
Taiwan_test, d = Taiwan_test_df.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
X_train = Taiwan_train.drop(lab,axis=1)
y_train = Taiwan_train[lab]
X_test = Taiwan_test.drop(lab,axis=1)
y_test = Taiwan_test[lab]
biasedLogModel = LogisticRegression(random_state=101)
biasedRfcModel = RandomForestClassifier(n_estimators=100,max_depth=4,random_state=101)
biasedLogModel.fit(X_train, y_train) 
biasedRfcModel.fit(X_train, y_train) 
print(f"Logistic regression validation accuracy: {biasedLogModel.score(X_test, y_test)}")
print(f"Random Forest       validation accuracy: {biasedRfcModel.score(X_test, y_test)}")

Logistic regression validation accuracy: 0.8162025316455697
Random Forest       validation accuracy: 0.8129957805907173


In [8]:
# Create the binary label dataset metric class for the train dataset
metric_train_bld = BinaryLabelDatasetMetric(Taiwan_train_df, 
                                            unprivileged_groups=unprivileged_groups,
                                            privileged_groups=privileged_groups)
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_train_bld.mean_difference())
print('Number of instances           :', metric_train_bld.num_instances())
print("Base Rate                     :%f" % metric_train_bld.base_rate())
print('Consistency                   :', metric_train_bld.consistency())
print('Disparate Impact              :', metric_train_bld.disparate_impact())
print('Mean Difference               :', metric_train_bld.mean_difference())
print('Statistical Parity Difference :', metric_train_bld.statistical_parity_difference()) 
print('# of positives(privileged)    :', metric_train_bld.num_positives(privileged=True))
print('# of positives(non-privileged):', metric_train_bld.num_positives(privileged=False))
print('Total positive instances"     :', metric_train_bld.num_positives(privileged=True)+metric_train_bld.num_positives(privileged=False))
print('# of negatives(privileged)    :', metric_train_bld.num_negatives(privileged=True))
print('# of negatives(non-privileged):', metric_train_bld.num_negatives(privileged=False))
print('Total negative instances"     :', metric_train_bld.num_negatives(privileged=True)+metric_train_bld.num_negatives(privileged=False))
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_train_bld.mean_difference())

#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = 0.026856
Number of instances           : 23698.0
Base Rate                     :0.777281
Consistency                   : [0.78298591]
Disparate Impact              : 1.0352073335588141
Mean Difference               : 0.026856111476450395
Statistical Parity Difference : 0.026856111476450395
# of positives(privileged)    : 8329.0
# of positives(non-privileged): 10091.0
Total positive instances"     : 18420.0
# of negatives(privileged)    : 2590.0
# of negatives(non-privileged): 2688.0
Total negative instances"     : 5278.0


#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = 0.026856


In [9]:
print('Interval : Optimization objective value for the interval')
TR = LFR(unprivileged_groups = unprivileged_groups, privileged_groups = privileged_groups, verbose=1, seed=101)
TR = TR.fit(Taiwan_train_df)

Interval : Optimization objective value for the interval
250 14587.517700040711
500 13692.670529850097
750 13120.096292091463
1000 13072.429071134293
1250 13017.349198439519
1500 12285.98907817623
1750 12246.958212120666
2000 12204.69803057172
2250 12111.772536359136
2500 12010.495619424868
2750 11933.850366042401
3000 11894.893787741807
3250 11883.334103852361
3500 11895.817193449264
3750 11903.992928145868
4000 11881.908544189171
4250 11881.909985108605
4500 11894.651485318795
4750 11832.362477118042
5000 11676.914999054541


In [10]:
# Transform training data and align features
Taiwan_train_lfr = TR.transform(Taiwan_train_df)

In [11]:
# Determine the transformed model accuracy for Logistic Regression and Random Forest Classifiers
# Convert the scaled Binary Labelled Dataset to a pandas dataframe for consistency 
Taiwan_train, d = Taiwan_train_lfr.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
X_train = Taiwan_train.drop(lab,axis=1)
y_train = Taiwan_train[lab]
debiasedLogModel = LogisticRegression(random_state=101)
debiasedRfcModel = RandomForestClassifier(n_estimators=100,max_depth=4,random_state=101)
debiasedLogModel.fit(X_train, y_train) 
debiasedRfcModel.fit(X_train, y_train) 
print(f"Logistic regression validation accuracy: {debiasedLogModel.score(X_test, y_test)}")
print(f"Random Forest       validation accuracy: {debiasedRfcModel.score(X_test, y_test)}")

Logistic regression validation accuracy: 0.7969620253164557
Random Forest       validation accuracy: 0.7939240506329114


In [12]:
display(Markdown("#### Transformed training dataset"))
from sklearn.metrics import classification_report
thresholds = [0.1, 0.3, 0.35, 0.4, 0.5, 0.6, 0.7]
for threshold in thresholds:
    
    # Transform training data and align features
    Taiwan_train_lfr = TR.transform(Taiwan_train_df,threshold=threshold)
    metric_train_lfr = BinaryLabelDatasetMetric(Taiwan_train_lfr, 
                                                 unprivileged_groups=unprivileged_groups,
                                                 privileged_groups=privileged_groups)
    print("Classification threshold = %f" % threshold)
    #print(classification_report(dataset_orig_train.labels, dataset_transf_train.labels))
    print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_train_lfr.mean_difference())

#### Transformed training dataset

Classification threshold = 0.100000
Difference in mean outcomes between unprivileged and privileged groups = -0.000090
Classification threshold = 0.300000
Difference in mean outcomes between unprivileged and privileged groups = -0.000706
Classification threshold = 0.350000
Difference in mean outcomes between unprivileged and privileged groups = -0.000752
Classification threshold = 0.400000
Difference in mean outcomes between unprivileged and privileged groups = -0.000537
Classification threshold = 0.500000
Difference in mean outcomes between unprivileged and privileged groups = 0.007184
Classification threshold = 0.600000
Difference in mean outcomes between unprivileged and privileged groups = 0.013575
Classification threshold = 0.700000
Difference in mean outcomes between unprivileged and privileged groups = 0.034784


In [13]:
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_train_bld.mean_difference())
print('Number of instances           :', metric_train_lfr.num_instances())
print("Base Rate                     :%f" % metric_train_lfr.base_rate())
print('Consistency                   :', metric_train_lfr.consistency())
print('Disparate Impact              :', metric_train_lfr.disparate_impact())
print('Mean Difference               :', metric_train_lfr.mean_difference())
print('Statistical Parity Difference :', metric_train_lfr.statistical_parity_difference()) 
print('# of positives(privileged)    :', metric_train_lfr.num_positives(privileged=True))
print('# of positives(non-privileged):', metric_train_lfr.num_positives(privileged=False))
print('Total positive instances"     :', metric_train_lfr.num_positives(privileged=True)+metric_train_lfr.num_positives(privileged=False))
print('# of negatives(privileged)    :', metric_train_lfr.num_negatives(privileged=True))
print('# of negatives(non-privileged):', metric_train_lfr.num_negatives(privileged=False))
print('Total negative instances"     :', metric_train_lfr.num_negatives(privileged=True)+metric_train_lfr.num_negatives(privileged=False))
display(Markdown("#### Original training dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_train_lfr.mean_difference())

Difference in mean outcomes between unprivileged and privileged groups = 0.026856
Number of instances           : 23698.0
Base Rate                     :0.877627
Consistency                   : [0.99774665]
Disparate Impact              : 1.0404994892335342
Mean Difference               : 0.03478379064310688
Statistical Parity Difference : 0.03478379064310688
# of positives(privileged)    : 9378.0
# of positives(non-privileged): 11420.0
Total positive instances"     : 20798.0
# of negatives(privileged)    : 1541.0
# of negatives(non-privileged): 1359.0
Total negative instances"     : 2900.0


#### Original training dataset

Difference in mean outcomes between unprivileged and privileged groups = 0.034784


In [14]:
display(Markdown("#### Individual fairness metrics"))
print("Consistency of labels in transformed training dataset= %f" %metric_train_lfr.consistency())
print("Consistency of labels in original training dataset= %f" %metric_train_bld.consistency())

#### Individual fairness metrics

Consistency of labels in transformed training dataset= 0.997747
Consistency of labels in original training dataset= 0.782986


In [15]:
# Compare the original and transformed datasets
#Convert the returned Binary Labelled Dataset to a pandas dataframe 
Taiwan_orig_df, d = Taiwan_train_df.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
Taiwan_lfr_df, d = Taiwan_train_lfr.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
# Check whether the transform on the original dataset has worked. 
# A false means that the dataset is transformed.
Taiwan_lfr_df.equals(Taiwan_df)

False

In [16]:
Taiwan_lfr_df.head(3)

,LIMIT_BAL,GENDER,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,DEFAULT
25872,0.260737,0.861354,0.369755,0.0,0.263271,-0.425176,-0.137054,-0.045944,-0.019097,-0.011862,...,0.181031,0.100390,0.186072,-0.065181,-0.251796,-0.173596,-0.195305,-0.108884,-0.132950,1.0
4201,0.259493,0.776667,0.433451,0.0,0.307023,-0.052166,0.081197,0.250589,0.162890,0.110407,...,0.258648,0.213249,0.280587,0.102961,0.029241,-0.085794,0.017126,0.036234,0.040384,1.0
12158,0.276893,0.794766,0.409435,1.0,0.305462,-0.127380,0.056890,0.175129,0.139355,0.077046,...,0.257858,0.175801,0.275315,0.072262,-0.015289,-0.091916,-0.032777,0.016895,0.007205,1.0


In [17]:
Taiwan_orig_df.head(3)

,LIMIT_BAL,GENDER,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,DEFAULT
25872,0.313131,1.0,0.333333,0.0,0.185185,0.0,0.0,0.0,0.0,0.0,...,0.163099,0.082278,0.180211,0.007341,0.004406,0.003523,0.003108,0.004513,0.000000,1.0
4201,0.020202,1.0,0.333333,0.0,0.537037,0.2,0.2,0.4,0.2,0.2,...,0.187232,0.109783,0.203327,0.009236,0.000000,0.001109,0.001980,0.003310,0.001430,1.0
12158,0.040404,1.0,0.666667,1.0,0.574074,0.2,0.2,0.2,0.2,0.2,...,0.177495,0.099301,0.195031,0.003154,0.001060,0.000735,0.001289,0.001817,0.001105,1.0


In [18]:
Taiwan_orig_df = Taiwan_orig_df.reset_index(drop=True)
Taiwan_lfr_df  = Taiwan_lfr_df.reset_index(drop=True)
TaiwanBool     = (Taiwan_orig_df != Taiwan_lfr_df).stack()  # Create Frame of comparison booleans
Taiwandiff     = pd.concat([Taiwan_orig_df.stack()[TaiwanBool], Taiwan_lfr_df.stack()[TaiwanBool]], axis=1)
Taiwandiff.columns=["Taiwan_df", "Taiwan_lfr"]
print(Taiwandiff)

                 Taiwan_df  Taiwan_lfr
0     LIMIT_BAL   0.313131    0.260737
      GENDER      1.000000    0.861354
      EDUCATION   0.333333    0.369755
      AGE         0.185185    0.263271
      PAY_1       0.000000   -0.425176
      PAY_2       0.000000   -0.137054
      PAY_3       0.000000   -0.045944
      PAY_4       0.000000   -0.019097
      PAY_5       0.000000   -0.011862
      PAY_6       0.000000   -0.144462
      BILL_AMT1   0.149718    0.043919
      BILL_AMT2   0.067747    0.066114
      BILL_AMT3   0.160686    0.138854
      BILL_AMT4   0.163099    0.181031
      BILL_AMT5   0.082278    0.100390
      BILL_AMT6   0.180211    0.186072
      PAY_AMT1    0.007341   -0.065181
      PAY_AMT2    0.004406   -0.251796
      PAY_AMT3    0.003523   -0.173596
      PAY_AMT4    0.003108   -0.195305
      PAY_AMT5    0.004513   -0.108884
      PAY_AMT6    0.000000   -0.132950
1     LIMIT_BAL   0.020202    0.259493
      GENDER      1.000000    0.776667
      EDUCATION   0.33333

In [19]:
## PCA Analysis of consitency

In [20]:
# At this stage the transformed dataframe will have the last threshold encountered!
print('this is the threshold being used :',threshold)
feat_cols = Taiwan_train_df.feature_names

orig_df = pd.DataFrame(Taiwan_train_df.features,columns=feat_cols)
orig_df['label'] = Taiwan_train_df.labels
orig_df['label'] = orig_df['label'].apply(lambda i: str(i))

transf_df = pd.DataFrame(Taiwan_train_lfr.features,columns=feat_cols)
transf_df['label'] = Taiwan_train_lfr.labels
transf_df['label'] = transf_df['label'].apply(lambda i: str(i))

this is the threshold being used : 0.7


In [21]:
from sklearn.decomposition import PCA

orig_pca = PCA(n_components=3)
orig_pca_result = orig_pca.fit_transform(orig_df[feat_cols].values)

orig_df['pca-one'] = orig_pca_result[:,0]
orig_df['pca-two'] = orig_pca_result[:,1] 
orig_df['pca-three'] = orig_pca_result[:,2]

display(Markdown("#### Original training dataset"))
print('Explained variation per principal component:')
print(orig_pca.explained_variance_ratio_)

#### Original training dataset

Explained variation per principal component:
[0.36911066 0.33736746 0.10051436]


In [22]:
transf_pca = PCA(n_components=3)
transf_pca_result = transf_pca.fit_transform(transf_df[feat_cols].values)

transf_df['pca-one'] = transf_pca_result[:,0]
transf_df['pca-two'] = transf_pca_result[:,1] 
transf_df['pca-three'] = transf_pca_result[:,2]

display(Markdown("#### Transformed training dataset"))
print('Explained variation per principal component:')
print(transf_pca.explained_variance_ratio_)

#### Transformed training dataset

Explained variation per principal component:
[0.94288752 0.0522229  0.00488949]


In [23]:
# Test whether we can predict the Sensitive (Protected) variable from the transformed training dataset
X_train = Taiwan_lfr_df.drop(protected_attr,axis=1)
y_train = Taiwan_lfr_df[protected_attr]
debiasedLogModel = LogisticRegression(random_state=101)
debiasedRfcModel = RandomForestClassifier(n_estimators=100,max_depth=4,random_state=101)
debiasedLogModel.fit(X_train, y_train) 
debiasedRfcModel.fit(X_train, y_train) 
# Now test whether we can predict Gender from the test dataset
Taiwan_test, d = Taiwan_test_df.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
X_test = Taiwan_test.drop(protected_attr,axis=1)
y_test = Taiwan_test[protected_attr]
print(f"Logistic regression validation accuracy: {debiasedLogModel.score(X_test, y_test)}")
print(f"Random Forest       validation accuracy: {debiasedRfcModel.score(X_test, y_test)}")

Logistic regression validation accuracy: 0.47139240506329116
Random Forest       validation accuracy: 0.5510548523206751


In [24]:
###Load, clean up original test data and compute metric

In [25]:
display(Markdown("#### Testing Dataset shape"))
print(Taiwan_test_df.features.shape)

metric_test_bld = BinaryLabelDatasetMetric(Taiwan_test_df, 
                                            unprivileged_groups=unprivileged_groups,
                                            privileged_groups=privileged_groups)
display(Markdown("#### Original test dataset"))
print("Difference in mean outcomes between unprivileged and privileged groups = %f" % metric_test_bld.mean_difference())

#### Testing Dataset shape

(5925, 23)


#### Original test dataset

Difference in mean outcomes between unprivileged and privileged groups = 0.019794


In [26]:
###Transform test data and compute metric

In [27]:
Taiwan_test_lfr = TR.transform(Taiwan_test_df, threshold=threshold)
metric_test_lfr = BinaryLabelDatasetMetric(Taiwan_test_lfr, 
                                         unprivileged_groups=unprivileged_groups,
                                         privileged_groups=privileged_groups)

In [28]:
print("Consistency of labels in tranformed test dataset= %f" %metric_test_lfr.consistency())

Consistency of labels in tranformed test dataset= 0.996759


In [29]:
print("Consistency of labels in original test dataset= %f" %metric_test_bld.consistency())

Consistency of labels in original test dataset= 0.789097


In [30]:
def check_algorithm_success():
    """Transformed dataset consistency should be greater than original dataset."""
    assert metric_test_lfr.consistency() > metric_test_bld.consistency(), "Transformed dataset consistency should be greater than original dataset."

print(check_algorithm_success())

None
